# Running and MNIST dataset classifier through CUSTOM image endpoints
---
## Adding package to the classpath
First of all we need to add the main package to the classpath so that the notebook can load all the necessary libraries from konduit-serving into the Jupyter notebook kernel.

Classpaths can be considered similar to `site-packages` in the python ecosystem where each library that's to be imported to your code is loaded from.

In [1]:
%classpath add jar ../../konduit.jar

In [2]:
%%bash
less train.py

import tensorflow as tf

from keras.datasets import mnist


tensorflow_version = tf.__version__
print(tensorflow_version)

# Load data
train_data, test_data = mnist.load_data()
x_train, y_train = train_data
x_test, y_test = test_data

# Normalize
x_train = x_train / 255.0
x_test = x_test / 255.0


def get_model():
    inputs = tf.keras.layers.Input(shape=(28, 28), name="input_layer")
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(200, activation="relu")(x)
    x = tf.keras.layers.Dense(100, activation="relu")(x)
    x = tf.keras.layers.Dense(60, activation="relu")(x)
    x = tf.keras.layers.Dense(30, activation="relu")(x)
    outputs = tf.keras.layers.Dense(10, activation="softmax", name="output_layer")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer='sgd',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


def train(epochs=8):
    model = get_model()
    mode

In [3]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.pipeline.api.data.Data;
import ai.konduit.serving.pipeline.api.data.Image;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;
import ai.konduit.serving.pipeline.impl.format.JavaImageFactory;
import ai.konduit.serving.pipeline.registry.ImageFactoryRegistry;
import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import ai.konduit.serving.pipeline.util.ObjectMappers;
import ai.konduit.serving.pipeline.registry.NDArrayConverterRegistry;
import ai.konduit.serving.data.nd4j.format.ND4JConverters;

public class OCREndPoint implements Endpoint {

    private PipelineExecutor pipelineExecutor;

    public OCREndPoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor; 
        ImageFactoryRegistry.addFactory(new JavaImageFactory()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.Nd4jToSerializedConverter()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.SerializedToNd4jArrConverter());
    }

    public HttpMethod type() { return HttpMethod.POST; }

    public String path() { return "/infer"; }

    public List<String> consumes() { return Arrays.asList("application/octet-stream","multipart/form-data"); }

    public List<String> produces() { return Arrays.asList("application/json"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> {
            handler.vertx().executeBlocking(taskHandler -> {
                Data image = Data.empty();
                
                try {
                    image.put("image",Image.create(ImageIO.read(new File(handler.fileUploads().iterator().next().uploadedFileName()))));
                } catch (IOException e) {
                    e.printStackTrace();
                }

                Data exec = pipelineExecutor.exec(image);
                
                handler.response().end(ObjectMappers.toJson(exec.getNDArray("output_layer").getAs(float[].class)));
                taskHandler.complete();
            },resultHandler -> {
                if(resultHandler.failed()) {
                    if(resultHandler.cause() != null)
                        if(handler.vertx().exceptionHandler() != null)
                            handler.vertx().exceptionHandler().handle(resultHandler.cause());
                        else {
                            resultHandler.cause().printStackTrace();
                        }
                    else {
                        System.err.println("Failed to process classification endpoint async task. Unknown cause.");
                    }
                }
            });

        };
    }
}

ai.konduit.OCREndPoint

In [4]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.endpoint.HttpEndpoints;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Arrays;
import java.util.List;

public class OCREndPoints implements HttpEndpoints {

    @Override
    public List<Endpoint> endpoints(Pipeline pipeline, PipelineExecutor pipelineExecutor) {
        return Arrays.asList(new OCREndPoint(pipelineExecutor));
    }
}

ai.konduit.OCREndPoints

In [5]:
import java.net.URLClassLoader;
import java.net.URL;
import java.io.File;

import java.util.ArrayList;
import java.util.List;

import org.apache.commons.io.FileUtils;
import java.io.IOException;

import java.nio.charset.StandardCharsets;

URL[] urls = ((URLClassLoader) Class.forName("ai.konduit.serving.vertx.config.InferenceConfiguration").getClassLoader()).getURLs();
List<String> classpaths = new ArrayList<>();

for(URL url : urls) {
    String singleClassPath = new File(url.toURI()).getAbsolutePath();
    System.out.println(singleClassPath);
    classpaths.add(singleClassPath);
}

try {
    String output = String.join(File.pathSeparator, classpaths);
    File classpathOutputPath = new File("classpath");
    FileUtils.writeStringToFile(new File("classpath"), output, StandardCharsets.UTF_8);
    System.out.format("Saved %s at: %s%n", output, classpathOutputPath.getAbsolutePath());
} catch (IOException e) {
    e.printStackTrace();
}

/tmp/beaker8649738043919203863/outDir
/root/konduit/konduit.jar
Saved /tmp/beaker8649738043919203863/outDir:/root/konduit/konduit.jar at: /root/konduit/demos/3-keras-mnist/classpath


null

In [9]:
%%bash
java -cp $(cat classpath) ai.konduit.serving.cli.launcher.KonduitServingLauncher serve -id keras-mnist -c keras.json -rwm -b

Starting konduit server...
Using classpath: /tmp/beaker8649738043919203863/outDir:/root/konduit/konduit.jar
INFO: Running command /root/miniconda/jre/bin/java -Dkonduit.logs.file.path=/root/.konduit-serving/command_logs/keras-mnist.log -Dlogback.configurationFile=/tmp/logback-run_command_ee96cf2607d7483f.xml ai.konduit.serving.cli.launcher.KonduitServingLauncher run --instances 1 -s inference -c keras.json -Dserving.id=keras-mnist
For server status, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher list'
For logs, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher logs keras-mnist'



In [10]:
%%bash
konduit list


Listing konduit servers...

 #   | ID                             | TYPE       | URL                  | PID     | STATUS     
 1   | keras-mnist                    | inference  | localhost:45155      | 2068    | started    




In [11]:
%%bash
konduit logs keras-mnist --lines 100

00:52:51.819 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Processing configuration: /root/konduit/demos/3-keras-mnist/keras.json
00:52:51.825 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Replaced standard System.out and System.err PrintStreams with SLF4JPrintStreams
00:52:51.827 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Redirected System.out and System.err to SLF4J for this context
00:52:51.827 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Starting konduit server with an id of 'keras-mnist'
00:52:52.122 [vert.x-worker-thread-0] INFO  a.k.s.p.registry.PipelineRegistry - Loaded 28 PipelineStepRunnerFactory instances
00:52:52.240 [vert.x-worker-thread-0] INFO  org.nd4j.linalg.factory.Nd4jBackend - Loaded [CpuBackend] backend
00:52:52.241 [vert.x-worker-thread-0] ERROR o.n.common.config.ND4JClassLoading - Cannot find class [org.nd4j.linalg.jblas.JblasBackend] of provided class-loader.
00:52:52.241 [vert.x-worker-thread-0] ERROR o.n.common.config.ND4JClassLoading - Cannot 

In [12]:
%%html
<img src="test-image.jpg" alt="title">

In [13]:
%%bash
curl -s -H "Content-Type: multipart/form-data" -X POST -F 'image=@test-image.jpg' http://localhost:$(konduit inspect keras-mnist -q {port})/infer

[ 9.0376153E-7, 1.0595608E-8, 1.3115231E-5, 0.44657645, 6.748624E-12, 0.5524258, 1.848306E-7, 2.7652052E-9, 9.76023E-4, 7.5933513E-6 ]



In [14]:
%%bash
konduit stop keras-mnist

Stopping konduit server 'keras-mnist'
Application 'keras-mnist' terminated with status 0

